# Numba Basics

Numba는 Python 함수의 Just-In-Time 컴파일러입니다. 
Python 함수를 호출할 때 2배(단순 NumPy 작업)에서 100배(복잡한 Python 루프)까지 더 빠르게 실행되는 machine code equivalent로 변환합니다. 
이 노트북에서는 Numba를 사용하는 몇 가지 기본적인 예를 보여줍니다.

In [1]:
import numpy as np
import numba
from numba import jit

Numba의 버전을 확인을 하는 방법으로는 :

In [2]:
print(numba.__version__)

0.55.1


Numba는 Python *decorators*를 사용하여 Python 함수를 스스로 컴파일되는 함수로 변환합니다. 

가장 일반적인 Numba 데코레이터는 CPU에서 실행하기 위한 일반 함수를 생성하는 `@jit`입니다.

Numba는 NumPy 배열을 사용하는 numerical functions(수치 함수)에서 가장 잘 작동합니다. 다음은 예입니다.

In [3]:
@jit(nopython=True)
def go_fast(a): # Function is compiled to machine code when called the first time
    trace = 0.0
    # assuming square input matrix
    for i in range(a.shape[0]):   # Numba likes loops
        trace += np.tanh(a[i, i]) # Numba likes NumPy functions
    return a + trace              # Numba likes NumPy broadcasting

`nopython=True` 옵션을 사용하려면 함수를 완전히 컴파일해야 합니다(파이썬 인터프리터 호출이 완전히 제거되도록). 

그렇지 않으면 예외가 발생합니다. 이러한 예외는 일반적으로 Python보다 나은 성능을 달성하기 위해 수정해야 하는 함수의 위치를 나타냅니다. 

항상 `nopython=True`를 사용하는 것이 좋습니다.

함수는 아직 컴파일되지 않았습니다. 그렇게 하려면 함수를 호출해야 합니다.

In [4]:
x = np.arange(100).reshape(10, 10)
go_fast(x)

array([[  9.,  10.,  11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.],
       [ 19.,  20.,  21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.],
       [ 29.,  30.,  31.,  32.,  33.,  34.,  35.,  36.,  37.,  38.],
       [ 39.,  40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.,  48.],
       [ 49.,  50.,  51.,  52.,  53.,  54.,  55.,  56.,  57.,  58.],
       [ 59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,  67.,  68.],
       [ 69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,  78.],
       [ 79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.],
       [ 89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.],
       [ 99., 100., 101., 102., 103., 104., 105., 106., 107., 108.]])

함수가 처음 호출되었을 때 함수의 새 버전이 컴파일되고 실행되었습니다. 

다시 호출하면 이전에 생성된 함수가 다른 컴파일 단계 없이 실행됩니다.

In [5]:
go_fast(2*x)

array([[  9.,  11.,  13.,  15.,  17.,  19.,  21.,  23.,  25.,  27.],
       [ 29.,  31.,  33.,  35.,  37.,  39.,  41.,  43.,  45.,  47.],
       [ 49.,  51.,  53.,  55.,  57.,  59.,  61.,  63.,  65.,  67.],
       [ 69.,  71.,  73.,  75.,  77.,  79.,  81.,  83.,  85.,  87.],
       [ 89.,  91.,  93.,  95.,  97.,  99., 101., 103., 105., 107.],
       [109., 111., 113., 115., 117., 119., 121., 123., 125., 127.],
       [129., 131., 133., 135., 137., 139., 141., 143., 145., 147.],
       [149., 151., 153., 155., 157., 159., 161., 163., 165., 167.],
       [169., 171., 173., 175., 177., 179., 181., 183., 185., 187.],
       [189., 191., 193., 195., 197., 199., 201., 203., 205., 207.]])

Numba에서 컴파일된 함수를 벤치마킹하려면 컴파일 단계를 포함하지 않고 시간을 측정하는 것이 중요합니다. 주어진 함수의 컴파일은 각 입력 유형 세트에 대해 한 번만 발생하지만 함수는 여러 번 호출되기 때문입니다.

노트북에서 `%timeit` 매직 함수는 루프에서 여러 번 함수를 실행하여 짧은 함수의 실행 시간을 보다 정확하게 추정하기 때문에 사용하는 것이 가장 좋습니다.

In [6]:
%timeit go_fast(x)

687 ns ± 11.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


컴파일되지 않은 함수와 비교해 보겠습니다. 

Numba 컴파일 함수에는 원래의 컴파일되지 않은 Python 함수인 특별한 `.py_func` 속성이 있습니다. 먼저 동일한 결과가 나오는지 확인해야 합니다.

In [7]:
np.testing.assert_array_equal(go_fast(x), go_fast.py_func(x))

그리고 Python 버전의 속도를 테스트해보면 :

In [8]:
%timeit go_fast.py_func(x)

14 µs ± 104 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


원래 Python 함수는 Numba 컴파일 버전보다 20배 이상 느립니다. 

그러나 Numba 함수는 Numba에서 매우 빠르며 Python에서는 그리 빠르지 않은 명시적 루프를 사용했습니다. 

예제 함수는 매우 간단하므로 NumPy 배열 표현식만 사용하여 `go_fast`의 대체 버전을 만들 수 있습니다.

In [9]:
def go_numpy(a):
    return a + np.tanh(np.diagonal(a)).sum()

In [10]:
np.testing.assert_array_equal(go_numpy(x), go_fast(x))

In [11]:
%timeit go_numpy(x)

5.38 µs ± 34.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


NumPy 버전은 Python보다 2배 이상 빠르지만 여전히 Numba보다 10배 느립니다.

### Supported Python Features

Numba는 NumPy 배열과 함께 사용할 때 가장 잘 작동하지만 Numba는 기본적으로 다른  data types도 지원합니다.

* `int`, `float`
* `tuple`, `namedtuple`
* `list` (with some restrictions)
* ... and others.  See the [Reference Manual](https://numba.pydata.org/numba-doc/latest/reference/pysupported.html) for more details.

특히 tuple은 함수에서 여러 값을 반환하는 데 유용합니다 :

In [12]:
import random

@jit(nopython=True)
def spherical_to_cartesian(r, theta, phi):
    '''Convert spherical coordinates (physics convention) to cartesian coordinates'''
    sin_theta = np.sin(theta)
    x = r * sin_theta * np.cos(phi)
    y = r * sin_theta * np.sin(phi)
    z = r * np.cos(theta)
    
    return x, y, z # return a tuple
    
@jit(nopython=True)
def random_directions(n, r):
    '''Return ``n`` 3-vectors in random directions with radius ``r``'''
    out = np.empty(shape=(n,3), dtype=np.float64)
    
    for i in range(n):
        # Pick directions randomly in solid angle
        phi = random.uniform(0, 2*np.pi)
        theta = np.arccos(random.uniform(-1, 1))
        # unpack a tuple
        x, y, z = spherical_to_cartesian(r, theta, phi)
        out[i] = x, y, z
    
    return out

In [13]:
random_directions(10, 1.0)

array([[-0.44244363,  0.8936341 ,  0.07524451],
       [-0.28717133,  0.40382089, -0.86859733],
       [-0.60035786,  0.4327777 , -0.67251312],
       [ 0.00653346, -0.99607455, -0.08827682],
       [-0.51347917, -0.80963305,  0.28431226],
       [-0.17610513,  0.18927977,  0.96600215],
       [-0.82641207,  0.44104266, -0.35003495],
       [-0.39507264, -0.48280047, -0.78155059],
       [ 0.26175148, -0.82317004,  0.50386233],
       [-0.11216924, -0.60022383, -0.79192766]])

Numba는 Python을 기계어로 번역할 때 [LLVM](https://llvm.org/) 라이브러리를 사용하여 대부분의 최적화 및 최종 코드 생성을 수행합니다. 

이것은 자동으로 생각할 필요도 없는 광범위한 최적화를 가능하게 합니다. 이전 임의 방향 예제에 대한 컴파일러의 출력을 검사하면 다음을 찾을 수 있습니다.

* 'spherical_to_cartesian()'의 함수 본문은 'random_directions'의 for 루프 본문에 직접 인라인되어 함수 호출의 오버헤드를 제거했습니다.
* `sin()` 및 `cos()`에 대한 개별 호출이 내부 `sincos()` 함수에 대한 단일 호출로 결합되었습니다.

이러한 종류의 교차 기능 최적화는 Numba가 컴파일된 NumPy 코드보다 성능이 더 좋을 수 있는 이유 중 하나입니다.